In [ ]:
# ------- PARAMETERS ------- # 

root = '/pnfs/pic.es/data/cta/LST/LST2/Data/ZFITS/'

runBackground = 16  # run of pedestal background that will be used to obtain the baseline of the pedestals

# -------------------------- #

# other parameters
dir_output = 'file_output/'
dir_graphs = 'graphs/'
minNumData = 800 # if in some pixel we have less, not taken into account in order to have some statistics


In [ ]:
import numpy             as np
import pandas            as pd
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import os
from ctapipe.instrument    import CameraGeometry
from ctapipe.visualization import CameraDisplay
from ctapipe.coordinates   import EngineeringCameraFrame
from traitlets.config      import Config
from ctapipe.io            import EventSource

import createCSV as csv
import auxiliar  as aux
aux.parameters()

# extracting the number of LST
LST_camera = aux.find_LST_num(root) 

# extracting geometry
date, subruns = aux.search(root,runBackground)    
config = Config(
    {'LSTEventSource': {'default_trigger_type': 'ucts','allowed_tels': [1],'min_flatfield_adc': 3000,
                        'min_flatfield_pixel_fraction': 0.8,},})  

path     = root+date+'/'+'LST-'+LST_camera+'.1.Run'+str(runBackground).zfill(5)+'.'+str(0).zfill(4)+'.fits.fz'
source   = EventSource(input_url=path, config=config)
camgeom1 = source.subarray.tel[1].camera.geometry

# geometry of CaCo 
camgeom2 = CameraGeometry.from_name("LSTCam")

# creating the folder if dont exist
aux.create_folder(dir_output)
aux.create_folder(dir_graphs)

## Read neighbors and values from the `.txt` files

In [ ]:
N_neighbor   = []
NN_neighbor  = []
MaxAmplitude = [[] for px in range(1855)]

for pixel in range(1855):
    
    if pixel % 160 == 0 or pixel == 0:
        print('Reading Pixels ' + str(int(100 * pixel / 1855)) + '%...')
        
    filename = dir_output + 'Pixel' + str(pixel).zfill(4) + '.txt'

    with open(filename) as file:
        lines = file.readlines()

    lines = [lines[i].replace('\n', '') for i in range(len(lines))]

    N_neighbor.append([int(lines[1].replace('[', '').replace(']', '').split(',')[i]) 
                       for i in range(len(lines[1].split(',')))])
    NN_neighbor.append([pixel] + [ int(lines[3].replace('[', '').replace(']', '').split(',')[i]) 
                                for i in range(len(lines[3].split(',')))])
 
    lines = lines[4:]
    if lines != []:
        
        for l in range(len(lines)):

            array = [int(lines[l].replace('[', '').replace(']', '').split(',')[i]) 
                     for i in range(len(lines[l].split(',')))]
            MaxAmplitude[pixel].append(array)
            
print('Reading pixels 100%')            

Print the data we obtained

In [ ]:
print('Pixel\t#Ev\tMaxAmplitud')

for px in range(1855):
    if px % 10 == 0:
        print('\n')
        
    maxs = [max(MaxAmplitude[px][i]) for i in range(len(MaxAmplitude[px])) if MaxAmplitude[px][i]!=[]]
    maximum = '--'
    if maxs != []:
        maximum = max(maxs)
    
    print(str(px) + '\t' + str(len(MaxAmplitude[px])) + '\t' + str(maximum))

### Calculating the pedestal of pixels in a run withut flashes

In [ ]:
# first we need to create the file from the background run input
# checking if the file already exists
path  = dir_output + 'data_Run' + str(runBackground) + '_Subrun' + str(0) + '.csv'

if os.path.exists(path) == False:
    # if dont exist we create the file
    print('Creating CSV file of the background run, will take a few minutes ...\n')
    
    csv.create(runBackground, root, dir_output)

    
# mean values extraction
print('\nReading CSV...')
df  = pd.read_csv(path)

pedestal = []

for i in range(1855): 
    pedestal.append(df.iloc[:, i * 2 + 1])

del df  # deleting to clean memory
print('Finished\n')

# mean of the pedestals for each pixel
print('Calculating means...\n')
TOTALMEAN  = [np.mean(pedestal[px]) for px in range(1855)]

# calculate neighbors of cluster
C_neighbor = aux.neighborCluster(camgeom1, camgeom2)

In [ ]:
# calculating mean values for each pixel and each neighbor - NN case
print('Calculating the NextNeighbors cases\n')
MeanAmplitudesNN = []

for px in range(1855):    
    
    MeanAmplitudestmp = []
    if len(MaxAmplitude[px]) > minNumData:
        
        matrix = np.transpose(MaxAmplitude[px])
        for nei in range(len(NN_neighbor[px])):
                MeanAmplitudestmp.append(np.mean(matrix[nei]))

    MeanAmplitudesNN.append(MeanAmplitudestmp)    
    
errorNN = []
for px in range(len(MeanAmplitudesNN)):
    
    for nei in range(len(MeanAmplitudesNN[px])):
        if (nei != 0) and (NN_neighbor[px][nei] not in N_neighbor[px]):
            
            errorNN.append(abs(100 * (MeanAmplitudesNN[px][nei] - TOTALMEAN[NN_neighbor[px][nei]]) / 
                               abs(MeanAmplitudesNN[px][0] - TOTALMEAN[px])))

                
# calculating mean values for each pixel and each neighbor - N case
MeanAmplitudesN = [] 
print('Calculating the Neighbors cases\n')
for px in range(1855):    
    
    MeanAmplitudestmp = []
    if len(MaxAmplitude[px]) > minNumData:
        
        matrix = np.transpose(MaxAmplitude[px])
        for nei in range(len(NN_neighbor[px])):
            
            if (NN_neighbor[px][nei] in N_neighbor[px]) or (nei == 0):
                MeanAmplitudestmp.append(np.mean(matrix[nei]))

    MeanAmplitudesN.append(MeanAmplitudestmp)    
    
errorN = []
for px in range(len(MeanAmplitudesN)):
    
    for nei in range(len(MeanAmplitudesN[px])):
        if nei != 0:
            
            errorN.append(abs(100 * (MeanAmplitudesN[px][nei] - TOTALMEAN[N_neighbor[px][nei-1]]) / 
                              abs(MeanAmplitudesN[px][0] - TOTALMEAN[px])))

# calculating mean values for each pixel and each neighbor - Cluster case
MeanAmplitudesC = []
print('Calculating the Cluster neighbor cases...\n')
for px in range(1855):    
    
    MeanAmplitudestmp = []
    if len(MaxAmplitude[px]) > minNumData:
        
        matrix = np.transpose(MaxAmplitude[px])
        for nei in range(len(NN_neighbor[px])):
            
            if (NN_neighbor[px][nei] in C_neighbor[px]) or (nei == 0):
                MeanAmplitudestmp.append(np.mean(matrix[nei]))

    MeanAmplitudesC.append(MeanAmplitudestmp)    
    
errorC = []
for px in range(len(MeanAmplitudesC)):
    
    for nei in range(len(MeanAmplitudesC[px])):
        if nei != 0:
            
            errorC.append(abs(100 * (MeanAmplitudesC[px][nei] - TOTALMEAN[C_neighbor[px][nei-1]]) / 
                              abs(MeanAmplitudesC[px][0] - TOTALMEAN[px])))


In [ ]:
fig,ax = plt.subplots(figsize=(12,5))       

ax.hist(errorNN, bins=33, density=True, alpha=0.8, histtype='stepfilled', lw=2.5, color='royalblue',
        label='Next-Nearest neighbor', zorder=0)
ax.hist(errorN , bins=29, density=True, alpha=1,   histtype='step',       lw=2.5, color='darkblue' ,
        label='Next neighbor')
ax.hist(errorC , bins=31, density=True, alpha=1,   histtype='step',       lw=2.5, color='deeppink' ,
        label='Cluster neighbor')

ax.legend()
ax.set_xlabel('%Crosstalk')
ax.set_ylabel('Normalized counts')

plt.savefig(dir_graphs + 'crosstalk_hist.pdf', format='pdf', bbox_inches='tight')  

plt.show()

Camera example of neighbors distribution

In [ ]:
# empty arrays
data   = [0 for i in range(1855)]
dataNN = [0 for i in range(1855)]
dataN  = [0 for i in range(1855)]
dataC  = [0 for i in range(1855)]
dataPX = [0 for i in range(1855)]

pixel         = 300
dataPX[pixel] = 1

for index in NN_neighbor[pixel]:
    if index not in N_neighbor[pixel]:
        dataNN[index] = 1

pixel         = 1750
dataPX[pixel] = 1

for i in N_neighbor[pixel]:
    dataN[i] = 1


pixel         = 1000
dataPX[pixel] = 1

for i in C_neighbor[pixel]:
    dataC[i] = 1

fig, ax = plt.subplots(figsize=(10, 8))
camgeom = source.subarray.tel[1].camera.geometry

cmap0      = colors.LinearSegmentedColormap.from_list("", ["#e9e9e9","#e9e9e9"])
camdisplay = CameraDisplay(camgeom.transform_to(EngineeringCameraFrame()),ax=ax,
                          image=data, show_frame=False, cmap=cmap0, title='') 

cmap_o = np.zeros([256, 4])
cmap_o[:, 3] = np.linspace(0, 1,    256)
cmap_o[:, 2] = np.linspace(0, 1,    256)
cmap_o[:, 1] = np.linspace(0, 0,    256)
cmap_o[:, 0] = np.linspace(0, 0.41, 256)
cmap_o     = colors.ListedColormap(cmap_o)
camdisplay = CameraDisplay(camgeom.transform_to(EngineeringCameraFrame()), ax=ax,
                           image=dataNN, show_frame=False, cmap=cmap_o, title='') 
plt.plot([], [], 's', color=(0.41,0,1), label='NN neighbor')

cmap_o = np.zeros([256, 4])
cmap_o[:, 3] = np.linspace(0, 1,    256)
cmap_o[:, 2] = np.linspace(0, 1,    256)
cmap_o[:, 1] = np.linspace(0, 0,    256)
cmap_o[:, 0] = np.linspace(0, 0.91, 256)
cmap_o     = colors.ListedColormap(cmap_o)
camdisplay = CameraDisplay(camgeom.transform_to(EngineeringCameraFrame()), ax=ax,
                           image=dataN, show_frame=False, cmap=cmap_o, title='') 
plt.plot([], [], 's', color=(0.91, 0, 1), label='N neighbor')


cmap_o = np.zeros([256, 4])
cmap_o[:, 3] = np.linspace(0, 1,    256)
cmap_o[:, 2] = np.linspace(0, 0,    256)
cmap_o[:, 1] = np.linspace(0, 0.83, 256)
cmap_o[:, 0] = np.linspace(0, 1,    256)
cmap_o     = colors.ListedColormap(cmap_o)
camdisplay = CameraDisplay(camgeom.transform_to(EngineeringCameraFrame()),ax=ax,
                           image=dataC,show_frame=False,cmap=cmap_o,title='') 
plt.plot([], [], 's', color=(1, 0.83, 0), label='Cluster neigh')


cmap_k = np.zeros([256, 4])
cmap_k[:, 3] = np.linspace(0, 1, 256)
cmap_k[:, 2] = np.linspace(0, 0, 256)
cmap_k[:, 1] = np.linspace(0, 0, 256)
cmap_k[:, 0] = np.linspace(0, 0, 256)
cmap_k     = colors.ListedColormap(cmap_k)
camdisplay = CameraDisplay(camgeom.transform_to(EngineeringCameraFrame()), ax=ax,
                           image=dataPX, show_frame=False, cmap=cmap_k, title='') 
plt.plot([], [], 's', color=(0, 0, 0), label='Target pixels')

ax.set_axis_off()
ax.legend()

plt.savefig(dir_graphs + 'example_neighbors.pdf', format='pdf', bbox_inches='tight')  
plt.show()